In [2]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px


In [3]:
df = pd.read_excel("C:/Users/andre/Downloads/Annuaire Alumni ABC .xlsx", sheet_name=0, header=0)
##a changer, prendre les données dans le drive

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nom                     265 non-null    object 
 1   Prénom                  265 non-null    object 
 2   Poste actuel            214 non-null    object 
 3   Entreprises             216 non-null    object 
 4   Secteur Public/Privé    221 non-null    object 
 5   Secteur d'activité      219 non-null    object 
 6   Ville de résidence      204 non-null    object 
 7   Pays de résidence       221 non-null    object 
 8   Coordonnées             99 non-null     object 
 9   Email                   227 non-null    object 
 10  Email secondaire        7 non-null      object 
 11  Mandat                  156 non-null    object 
 12  Ancien(s) poste(s) ABC  121 non-null    object 
 13  Linkedin                228 non-null    object 
 14  Ecole                   222 non-null    ob

In [5]:

# Créer l'application Dash avec les styles Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Mise en page de l'application Dash
app.layout = dbc.Container(
    [
        dbc.NavbarSimple(
            brand="Réseau: Suivi des Alumnis",
            brand_href="#",
            color="primary",
            dark=True,
            style={'background-color': 'lightgreen', 'padding-left': '20px'},  # Couleur de fond verte pour la barre de navigation
        ),
        dcc.Tabs([
            dcc.Tab(label='Affichage par Pays de résidence', children=[
                html.Div([
                    html.Div(
                        dcc.Dropdown(
                            id='dropdown-country',
                            options=[{'label': country if pd.notna(country) else 'Non renseigné', 'value': country if pd.notna(country) else 'Non renseigné'} for country in df['Pays de résidence'].unique()],
                            value=None,
                            placeholder="Sélectionnez un pays",
                            className="mt-3",
                            style={'background-color': 'lightgreen', 'width': '300px', 'margin': 'auto'},  # Couleur de fond verte pour le dropdown
                        ), style={'text-align': 'center', 'margin-top': '20px'}
                    ),
                    html.Div(id='output-container', className="mt-4", style={'padding': '20px', 'background-color': 'white'}),
                ], className="jumbotron", style={'background-color': 'white', 'border': '1px solid lightgray', 'border-radius': '5px'})
            ]),
            dcc.Tab(label='Statistiques', children=[
                html.Div([
                    html.H3("Statistiques", className="display-4"),
                    html.Div(id='statistics-output', className="mt-4", style={'padding': '20px', 'background-color': 'white'}),
                ], className="jumbotron", style={'background-color': 'white', 'border': '1px solid lightgray', 'border-radius': '5px'})
            ]),
            dcc.Tab(label='Graphique interactif', children=[
                html.Div([
                    html.H3("Graphique interactif", className="display-4"),
                    dcc.Graph(id='interactive-graph'),  # Premier graphique
                    dcc.Graph(id='interactive-graph-2'),  # Deuxième graphique ajouté ici
                ], className="jumbotron", style={'background-color': 'white', 'border': '1px solid lightgray', 'border-radius': '5px'})
            ])
        ])
    ],
    fluid=True,
)

# Ajouter des styles CSS personnalisés
app.css.append_css({
    'external_url': 'https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css'
})

# Callback pour afficher les personnes par pays de résidence
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-country', 'value')]
)
def display_people_by_country(selected_country):
    if selected_country is not None:
        if selected_country == 'Non renseigné':
            filtered_df = df[pd.isna(df['Pays de résidence'])]
        else:
            filtered_df = df[df['Pays de résidence'] == selected_country]
        
        people_info = []
        for index, row in filtered_df.iterrows():
            person_info = html.Div([
                html.P(f"Nom: {row['Nom'] if pd.notna(row['Nom']) else 'Non renseigné'}"),
                html.P(f"Prénom: {row['Prénom'] if pd.notna(row['Prénom']) else 'Non renseigné'}"),
                html.P(f"Poste actuel: {row['Poste actuel'] if pd.notna(row['Poste actuel']) else 'Non renseigné'}"),
                html.P(f"Email: {row['Email'] if pd.notna(row['Email']) else 'Non renseigné'}"),
                html.P(f"Ancien(s) poste(s) ABC: {row['Ancien(s) poste(s) ABC'] if pd.notna(row['Ancien(s) poste(s) ABC']) else 'Non renseigné'}"),
                html.P(f"Mandat ABC: {row['Mandat'] if pd.notna(row['Mandat']) else 'Non renseigné'}")
            ], style={'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '10px'})
            people_info.append(person_info)
        
        phrase_nombre_personnes = f"Nombre de personnes résidant au {selected_country if selected_country != 'Non renseigné' else 'pays non renseigné'}: {len(filtered_df)}"
        
        return html.Div([
            html.H3("Affichage des personnes par pays de résidence", className="display-4"),
            html.P(phrase_nombre_personnes, className="lead"),
            html.Hr(),  
            html.Div(people_info)
        ], className="jumbotron custom-jumbotron")
    else:
        return html.Div()
    
# Callback pour afficher les statistiques
@app.callback(
    Output('statistics-output', 'children'),
    [Input('dropdown-country', 'value')]
)
def display_statistics(selected_country):
    if selected_country is None:
        return html.Div()

    if selected_country == 'Non renseigné':
        filtered_df = df[pd.isna(df['Pays de résidence'])]
    else:
        filtered_df = df[df['Pays de résidence'] == selected_country]

    # Statistiques par secteur d'activité
    sector_statistics = filtered_df['Secteur d\'activité'].value_counts().reset_index()
    sector_statistics.columns = ['Secteur d\'activité', 'Nombre de personnes']

    # Statistiques par ville de résidence
    city_statistics = filtered_df['Ville de résidence'].value_counts().reset_index()
    city_statistics.columns = ['Ville de résidence', 'Nombre de personnes']

    statistics_output = html.Div([
        html.H4("Statistiques pour le pays de résidence sélectionné", className="display-5"),
        html.P(f"Nombre de personnes résidant au {selected_country if selected_country != 'Non renseigné' else 'pays non renseigné'}: {len(filtered_df)}", className="lead"),
        html.Hr(),
        html.H5("Statistiques par secteur d'activité", className="display-6"),
        html.Table([
            html.Thead([
                html.Tr([html.Th(col) for col in sector_statistics.columns])
            ]),
            html.Tbody([
                html.Tr([
                    html.Td(sector_statistics.iloc[i][col]) for col in sector_statistics.columns
                ]) for i in range(len(sector_statistics))
            ])
        ]),
        html.Hr(),
        html.H5("Statistiques par ville de résidence", className="display-6"),
        html.Table([
            html.Thead([
                html.Tr([html.Th(col) for col in city_statistics.columns])
            ]),
            html.Tbody([
                html.Tr([
                    html.Td(city_statistics.iloc[i][col]) for col in city_statistics.columns
                ]) for i in range(len(city_statistics))
            ])
        ])
    ], className="jumbotron custom-jumbotron")

    return statistics_output

# Callback pour afficher le graphique interactif
@app.callback(
    Output('interactive-graph', 'figure'),
    [Input('dropdown-country', 'value')]
)
def display_interactive_graph(selected_country):
    if selected_country is None:
        return {}

    if selected_country == 'Non renseigné':
        filtered_df = df[pd.isna(df['Pays de résidence'])]
    else:
        filtered_df = df[df['Pays de résidence'] == selected_country]

    # Créer un graphique interactif avec Plotly Express
    fig = px.scatter(filtered_df, x='Secteur d\'activité', y='Ville de résidence', color='Secteur d\'activité', title=f'Distribution des intervenants par secteur d\'activité et ville de résidence ({selected_country})')

    return fig

# Callback pour afficher le deuxième graphique interactif
@app.callback(
    Output('interactive-graph-2', 'figure'),
    [Input('dropdown-country', 'value')]
)
def display_interactive_graph_2(selected_country):
    if selected_country is None:
        return {}

    if selected_country == 'Non renseigné':
        filtered_df = df[pd.isna(df['Pays de résidence'])]
    else:
        filtered_df = df[df['Pays de résidence'] == selected_country]

    # Compter le nombre d'intervenants par ville de résidence
    city_count = filtered_df['Ville de résidence'].value_counts().reset_index()
    city_count.columns = ['Ville de résidence', 'Nombre d\'intervenants']

    # Créer un deuxième graphique interactif avec Plotly Express
    fig = px.bar(city_count, x='Ville de résidence', y='Nombre d\'intervenants', text='Nombre d\'intervenants', 
                 title=f'Nombre d\'alumnis par ville de résidence ({selected_country})')

    # Personnaliser la mise en forme du texte au-dessus des barres
    fig.update_traces(texttemplate='%{text}', textposition='outside')

    return fig

if __name__ == '__main__':
    app.run_server(port=8052, debug=True)


c:\Users\andre\AppData\Local\Programs\Python\Python39\lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

